In [1]:
from __future__ import print_function
import os
import numpy as np
from numpy import newaxis
from sklearn import preprocessing
np.random.seed(1337)

import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Dropout
from keras.models import Model,Sequential
import sys

import pandas as pd
import random

Using TensorFlow backend.


# Reading in data

In [130]:
print("Data read started...")
data = pd.read_csv("nback_full_1.csv")
data = data.as_matrix()
print ("Data read finished.")

print(data.shape)

Data read started...
Data read finished.
(5057, 19)


# Eliminate EEG data

In [131]:
for i in range (1,5):
    data = np.delete(data, 1, 1) 
    

data = data[:,0:5]
data.shape

(5057, 5)

In [132]:
data[1]

array(['3back', 1802.4, 796.06, 39.528, 4.7517777777777805], dtype=object)

# Dictionary for the levels

In [133]:
level=["0back","1back","2back","3back"]
#level=["tetris1","tetris2","tetris3"]
level2int = dict((p, i) for i, p in enumerate(level))
int2level = dict((i, p) for i, p in enumerate(level))

# Z normalize

In [135]:
for i in range (1,5):
    data[:, i] = preprocessing.scale(data[:, i])

/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [136]:
for i in range(len(data)):
    data[i, 0] = level2int[data[i, 0]]

x_data = data[:, 1:]
y_data = data[:, 0]

# One-hot encoding

In [137]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y_one_hot = ohe.fit_transform(y_data.reshape(-1,1)).toarray()
print(y_one_hot)

[[ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 ..., 
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]]


# Shuffle data

In [138]:
indices = np.arange(x_data.shape[0])
np.random.shuffle(indices)

x_data = x_data[indices]
y_one_hot = y_one_hot[indices]

# Divide into train, validation and test data

In [139]:
len_data = len(x_data)

nb_test = int(len_data*0.15)
nb_validation = int(len_data*0.15)
nb_train = int(len_data*0.7)

end_valid = nb_train+nb_validation

x_train = x_data[0:nb_train]
y_train = y_one_hot[0:nb_train]

x_valid = x_data[nb_train:end_valid]
y_valid = y_one_hot[nb_train:end_valid]

x_test = x_data[end_valid:]
y_test = y_one_hot[end_valid:]

In [140]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(3539, 4) (3539, 4) (758, 4) (758, 4) (760, 4) (760, 4)


# Build the net

In [141]:
model = Sequential()
#TETRIS
#model.add(Dense(200, input_shape=(4,)))
#model.add(Dropout(0.25))
#model.add(Dense(200, activation='relu'))
#model.add(Dropout(0.25))
#model.add(Dense(700, activation='relu'))
#model.add(Dense(200, activation='relu'))
#model.add(Dense(200, activation='relu'))
#model.add(Dropout(0.25))
#model.add(Dense(200, activation='relu'))
#model.add(Dropout(0.25))
#model.add(Dense(3, activation='softmax'))

model.add(Dense(200, input_shape=(4,)))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))

In [142]:
batch_size = 500
epochs = 300
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='auto')

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(x_train, y_train, nb_epoch=epochs,batch_size=batch_size, 
          callbacks=[earlyStopping], shuffle=True, validation_data = (x_valid, y_valid))

Train on 3539 samples, validate on 758 samples
Epoch 1/300
3539/3539 [==============================] - 1s - loss: 1.3813 - acc: 0.2735 - val_loss: 1.3698 - val_acc: 0.3153
Epoch 2/300
3539/3539 [==============================] - 0s - loss: 1.3649 - acc: 0.3055 - val_loss: 1.3583 - val_acc: 0.3311
Epoch 3/300
3539/3539 [==============================] - 0s - loss: 1.3572 - acc: 0.3165 - val_loss: 1.3584 - val_acc: 0.3391
Epoch 4/300
3539/3539 [==============================] - 0s - loss: 1.3552 - acc: 0.3156 - val_loss: 1.3544 - val_acc: 0.3377
Epoch 5/300
3539/3539 [==============================] - 0s - loss: 1.3515 - acc: 0.3221 - val_loss: 1.3517 - val_acc: 0.3325
Epoch 6/300
3539/3539 [==============================] - 0s - loss: 1.3547 - acc: 0.3159 - val_loss: 1.3455 - val_acc: 0.3351
Epoch 7/300
3539/3539 [==============================] - 0s - loss: 1.3489 - acc: 0.3204 - val_loss: 1.3409 - val_acc: 0.3364
Epoch 8/300
3539/3539 [==============================] - 0s - loss: 1.3

In [143]:
model.evaluate(x_test, y_test, batch_size=batch_size)

760/760 [==============================] - 0s     


[1.0296555883006047, 0.53157895489742879]

In [144]:
from sklearn.metrics import classification_report,confusion_matrix

y_pred = model.predict_classes(x_test)
print(y_pred)

p=model.predict_proba(x_test)

target_names = ['class 0(0back)', 'class 1(1back)', 'class 2(2back)', 'class 3(3back)']
#target_names = ['class 0(tetris1)', 'class 1(tetris2)', 'class 2(tetris3)']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

760/760 [==============================] - 0s     
[1 1 3 1 2 2 0 2 2 2 1 1 2 2 2 2 2 1 2 1 0 3 3 0 3 2 3 0 2 2 2 0 2 2 1 2 0
 2 2 1 1 3 0 2 3 0 1 1 2 0 2 0 1 1 1 3 2 2 3 1 0 3 3 2 2 0 3 1 3 0 2 2 2 1
 0 2 1 0 0 2 1 1 3 1 3 0 3 0 0 2 2 1 1 1 2 1 3 2 0 2 2 2 1 0 2 1 1 3 1 3 2
 0 0 2 2 3 3 3 3 3 2 2 1 0 0 3 1 2 0 2 1 2 2 2 2 2 2 3 2 1 0 2 2 1 1 2 0 2
 2 2 0 1 1 0 1 2 2 2 0 0 0 1 2 1 3 1 2 0 1 0 1 3 0 0 1 0 0 1 0 2 2 2 1 1 2
 2 2 2 2 1 1 1 0 2 2 3 1 2 3 2 1 1 2 2 2 2 0 2 3 3 3 0 3 0 3 3 3 1 1 1 1 0
 2 1 2 2 0 1 0 2 2 2 2 1 2 0 1 3 2 0 1 3 3 3 2 1 1 1 3 0 2 0 0 3 0 2 3 0 0
 0 1 0 1 1 2 2 1 1 2 0 2 2 2 3 3 0 3 1 2 3 3 3 3 0 2 0 1 2 3 2 2 0 0 2 1 2
 1 3 3 1 2 0 1 0 2 0 3 2 3 3 3 2 2 1 1 2 3 1 2 3 3 0 2 1 1 3 1 1 2 1 1 3 3
 0 1 2 2 2 1 3 0 3 0 2 3 0 0 2 2 2 3 3 2 2 2 3 3 1 1 2 2 1 2 1 3 3 0 0 1 0
 3 1 1 1 2 2 0 1 0 3 1 3 3 0 3 3 1 3 1 0 3 3 2 0 1 0 3 1 2 2 0 2 0 1 1 3 1
 1 0 1 2 2 1 1 1 2 1 1 2 0 3 0 0 2 1 2 0 2 1 1 2 1 2 2 3 1 1 1 1 2 3 2 1 2
 1 0 1 3 1 2 1 1 0 1 3 1 0 2 3 3 1 2 2 1 2 1 1 3 